# DC Properties

In [61]:
import requests as r
import mechanize
from bs4 import BeautifulSoup as soup
import json
import pandas as pd

def clean_str(s):
    return s.replace("\n"," ").replace("\s*"," ").strip()

In [66]:
def getPages(br,ward=1,page=1):
    if page==1:
        resp = br.submit()
    else:
        resp = br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?page=%d" % page)
    html = soup(resp.read(),"html.parser")
    table = html.find("table",attrs={"class":"","border":"0","cellpadding":4,"cellspacing":2})
    if len(table)>0:
        i = 0
        rows=list()
        for tr in table.find_all("tr"):  
            if i==0:
                header = [clean_str(td.text) for td in tr.find_all("th")]
            else:
                row = [clean_str(td.text) for td in tr.find_all("td")]
                values = row
                row = { e[0]:e[1] for e in zip(header,values) }
                #print row
            rows.append(row)
            i+=1
        pd.DataFrame(rows,dtype=str).to_csv("ward%d_%03d.csv" % (ward,page) )
        getPages(br,ward,page+1)
    
        

In [67]:
# search by ward
br = mechanize.Browser()
br.open("https://www.taxpayerservicecenter.com/RP_Results.jsp?search_type=Sales")
br.select_form(name="SearchForm")
br["selectWard"]=["1"]
getPages(br)


UnboundLocalError: local variable 'row' referenced before assignment